<a href="https://colab.research.google.com/github/mithunkumarsr/codemix/blob/master/mBERT_Layer_POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.9 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.3 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=3b8a2679d82da73d985b93b7a705935fb2f7045ff69be8b41a673256a5b35778
  Stored in directory: /root/.cache/pip/wheels/e2/a5/92/2c80d1928733611c2747a9820e1324a6835524d9411510c142
Successfully built seqeval


In [ ]:
from datasets import load_dataset, ClassLabel, DatasetDict
from transformers import AutoModel, AutoTokenizer, DataCollatorForTokenClassification, AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch.nn as nn
import torch
import evaluate
from transformers import get_scheduler
from torch.optim import AdamW
from torch.utils.data import DataLoader


In [ ]:
lince_dataset_pos = load_dataset('lince', 'pos_spaeng')
lince_dataset_pos_test = lince_dataset_pos["test"]
lince_dataset_pos = DatasetDict({"train": lince_dataset_pos['train'], "validation":lince_dataset_pos["validation"]})
lince_dataset_pos.column_names

Generating train split:   0%|          | 0/27893 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4298 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10720 [00:00<?, ? examples/s]

Dataset lince downloaded and prepared to /root/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'train': ['idx', 'words', 'lid', 'pos'],
 'validation': ['idx', 'words', 'lid', 'pos']}

In [ ]:
labels_list = set()

for split in ['train', 'validation']:
    dataset = lince_dataset_pos[split]
    for datapoint in dataset:
        labels_list.update(datapoint['pos'])

labels_list = list(labels_list)

# %%
# Loading the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased', return_tensors='pt')
label_all_tokens = True
labels_encoder = ClassLabel(names = labels_list)

# Tokenizing and aligning the tags
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words"], truncation=True, is_split_into_words=True)
    examples["pos"] = [labels_encoder.str2int(tag) for tag in examples["pos"]]

    labels = []
    for i, label in enumerate(examples["pos"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# %%
# Tokenizing the dataset using map function
preprocessed_dataset = lince_dataset_pos.map(tokenize_and_align_labels, batched=True)
# preprocessed_dataset['train'].to_pandas()

# %%
preprocessed_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/27893 [00:00<?, ? examples/s]

Map:   0%|          | 0/4298 [00:00<?, ? examples/s]

In [ ]:
class mBERTLayerModel(nn.Module):
    def __init__(self, checkpoint, num_labels, layer_number ):
        super(mBERTLayerModel, self).__init__()
        self.num_labels = num_labels
        self.layer_number = layer_number
        self.model = AutoModel.from_pretrained(checkpoint, config = AutoConfig.from_pretrained(checkpoint,
                                                                                                       output_attention = True,
                                                                                                       output_hidden_states = True ) )
        self.dropout = nn.Dropout(0.1)
#        self.classifier = nn.LSTM(input_size=768, hidden_size=self.num_labels, num_layers=1, batch_first=True)
        self.classifier = nn.Linear(in_features=768, out_features=num_labels, bias=True)

    def forward(self, input_ids = None, attention_mask=None, labels = None ):
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)

        # last_hidden_state = outputs[0]

        bert_embeddings = outputs.hidden_states[self.layer_number]
        sequence_outputs = self.dropout(bert_embeddings)
        # print("dropout layer output: ", sequence_outputs.shape)
        logits = self.classifier(sequence_outputs)

        loss = None
        if labels is not None:
            loss_func = nn.CrossEntropyLoss()
            # print("Classifier output sent as is shape: ", logits[0].shape)
            loss = loss_func(torch.reshape(logits, (-1, self.num_labels)), labels.view(-1))
            # exit()
            return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)



data_collator = DataCollatorForTokenClassification(tokenizer)


Batch Size can be changed in the following cell

In [ ]:
train_dataloader = DataLoader(
    preprocessed_dataset["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    preprocessed_dataset["validation"], batch_size=16, collate_fn=data_collator
)

Layer number can be changed in the following cell

In [ ]:
lay_num = 3
POS_tagger = mBERTLayerModel(checkpoint='bert-base-multilingual-uncased', num_labels=len(labels_list), layer_number=lay_num)


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

optimizer = AdamW(POS_tagger.parameters(), lr=5e-5)

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

metric = evaluate.load("seqeval")

from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)))

for epoch in range(num_epochs):
    POS_tagger.train()
    for batch in train_dataloader:
        batch = {k: v for k, v in batch.items()}
        outputs = POS_tagger(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

    POS_tagger.eval()
    for batch in eval_dataloader:
        batch = {k: v for k, v in batch.items()}
        with torch.no_grad():
            outputs = POS_tagger(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        labels = batch['labels']
        true_predictions = [
            [labels_encoder.int2str(p.item()) for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [labels_encoder.int2str(l.item()) for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        metric.add_batch(predictions=true_predictions, references=true_labels)
        progress_bar_eval.update(1)
    print("===================== Epoch: ", epoch, " =====================")
    print(metric.compute())


1744


  0%|          | 0/1744 [00:00<?, ?it/s]

  0%|          | 0/269 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


===================== Epoch:  0  =====================


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: INTJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PUNCT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWa

{'ART': {'precision': 0.9673024523160763, 'recall': 0.9806629834254144, 'f1': 0.9739368998628258, 'number': 724}, 'CONJ': {'precision': 0.8672985781990521, 'recall': 0.8806544754571703, 'f1': 0.8739255014326647, 'number': 1039}, 'DJ': {'precision': 0.8122109158186864, 'recall': 0.8346007604562737, 'f1': 0.8232536333802157, 'number': 1052}, 'DP': {'precision': 0.9554973821989529, 'recall': 0.9495317377731529, 'f1': 0.9525052192066805, 'number': 1922}, 'DV': {'precision': 0.8733496332518338, 'recall': 0.8837209302325582, 'f1': 0.8785046728971962, 'number': 2021}, 'ERB': {'precision': 0.949187759727994, 'recall': 0.9564141606395128, 'f1': 0.9527872582480091, 'number': 5254}, 'ET': {'precision': 0.9519440803844473, 'recall': 0.971900089206066, 'f1': 0.9618185830942396, 'number': 2242}, 'NK': {'precision': 0.9960159362549801, 'recall': 1.0, 'f1': 0.998003992015968, 'number': 250}, 'NTJ': {'precision': 0.9369038884812912, 'recall': 0.918705035971223, 'f1': 0.9277152197602615, 'number': 1390}